In [1]:
from shapely import geometry
import numpy as np
import pandas as pd
import xarray as xr
from pathlib import Path

import sys
sys.path.append('./')
sys.path.append('../')
from phase_o_matic.phase_delay import calculate_wet_refractivity, calculate_refractive_indexes, get_delay

In [2]:
np.random.seed(1)
heights = np.linspace(-200, 40000, 300)
lats = np.linspace(43, 45, 10).astype('float32')
lons = np.linspace(-116, -115, 13).astype('float32')
test_ds = xr.Dataset(
    {
        'air_pressure': (['time','latitude', 'longitude','height'], 100*np.random.random((1, 10, 13, 300)).astype('float32'), {'units':'pascals', 'long_name':'pressure_level'}),
        'temperature': (['time','latitude', 'longitude','height'], np.random.random((1, 10, 13, 300)).astype('float32'), {'units' :'K', 'long_name' :'Temperature', 'standard_name' :'air_temperature'}),
        'vapor_pressure': (['time','latitude', 'longitude','height'], np.random.random((1, 10, 13, 300)).astype('float32'), {'units': 'pascals'}),
    },  
    coords = {
        "longitude" : (["longitude"], lons, {'units' :'degrees_east', 'long_name' :'longitude'}),
        "latitude" : (["latitude"], lats, {'units' :'degrees_north', 'long_name' :'latitude'}),
        "height" : (["height"], heights, {'units' : 'meters', 'long_name' :'geopotential_heights'}),
        "time" : (["time"], [pd.to_datetime('2020-01-04T09:00')], {'long_name' :'time'}),
    },
    attrs={'Conventions' :'CF-1.6', 'history' :'2023-05-04 17:03:14 GMT by grib_'}
)

test_ds = calculate_refractive_indexes(test_ds)

In [6]:
dem = xr.DataArray(np.linspace(1000, 3000, 200*300).reshape(200, 300),
                   coords = [np.linspace(43, 45, 200), np.linspace(-116, -115, 300)],
                   dims = ['latitude','longitude']
)

inc = xr.DataArray(np.linspace(0, np.deg2rad(90), 200*300).reshape(200, 300),
                   coords = [np.linspace(43, 45, 200), np.linspace(-116, -115, 300)],
                   dims = ['latitude','longitude']
)

# dem_ds = dem.to_dataset(name = 'dem')
# dem_ds['inc'] = inc